In [1]:
import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import os
import glob
from pathlib import Path
import SimpleITK as sitk # para calcular rangos
import numpy as np
import io
from PIL import Image
import random
import sys
import pandas as pd
import math
import pandas as pd
import sklearn
import sklearn.metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [2]:
sys.path.append("/Users/hugom/PET-IA/src")

In [3]:
from datasets import ADNIDataset, FleniMyriamDataset, BaseDataset
from transforms import TransformGridImage, ToLabelOutputConfigurable, ToLabelOutput
from train_lib import train_model, set_parameter_requires_grad, initialize_model
from util import collectAllData
import torch
import torchvision
from torchvision import transforms, utils, models, datasets

# Validación

In [7]:
config = {
  "last_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_3classes_0_epoch21.pth",
    "num_classes": 3,
    "validationSet": "Muestra3700_80_10_10_dxlast_val.csv",
    "testSet": "Muestra3700_80_10_10_dxlast_test.csv",
    "truthLabel": "DX_last",
    "mean": 0.26114351157369636,
    "std": 0.46353330246995716
  },
  "sev_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_3classes_0_epoch31.pth",
    "num_classes": 3,
    "validationSet": "Muestra3700_80_10_10_dxmost_severe_val.csv",
    "testSet": "Muestra3700_80_10_10_dxmost_severe_test.csv",
    "truthLabel": "DX_most_severe",
    "mean": 0.2608884478852832,
    "std": 0.4633194609974111
  },
  "visit_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_3classes_0_epoch10.pth",
    "num_classes": 3,
    "validationSet": "Muestra3700_80_10_10_dxvisit_val.csv",
    "testSet": "Muestra3700_80_10_10_dxvisit_test.csv",
    "truthLabel": "DX",
    "mean": 0.2611961435953144,
    "std": 0.46364206293844507
  },
  "last_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_2classes_0_epoch18.pth",
    "num_classes": 2,
    "validationSet": "Muestra3700_80_10_10_dxlast_val.csv",
    "testSet": "Muestra3700_80_10_10_dxlast_test.csv",
    "truthLabel": "DX_last",
    "mean": 0.26114351157369636,
    "std": 0.46353330246995716
  },
  "sev_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_2classes_0_epoch16.pth",
    "num_classes": 2,
    "validationSet": "Muestra3700_80_10_10_dxmost_severe_val.csv",
    "testSet": "Muestra3700_80_10_10_dxmost_severe_test.csv",
    "truthLabel": "DX_most_severe",
    "mean": 0.2608884478852832,
    "std": 0.4633194609974111
  },
  "visit_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_2classes_0_epoch5.pth",
    "num_classes": 2,
    "validationSet": "Muestra3700_80_10_10_dxvisit_val.csv",
    "testSet": "Muestra3700_80_10_10_dxvisit_test.csv",
    "truthLabel": "DX",
    "mean": 0.2611961435953144,
    "std": 0.46364206293844507
  }
}


In [4]:
def executeModel(mode, weights, num_classes, validationSet, testSet, truthLabel, mean, std):
    if mode == 'validation':
        csv = f"/Users/hugom/PET-IA/Sets/{validationSet}"
    elif mode == 'test':
        csv = f"/Users/hugom/PET-IA/Sets/{testSet}"
    else:
        print(f"Invalid mode {mode}")
    imagesFolder = '/Users/hugom/Tesis/Imagenes/ADNI-MUESTRA-FULL-stripped-preprocessed3'
    means = [mean, mean, mean]
    stds = [std, std, std]
    transforms = torchvision.transforms.Compose([
        TransformGridImage(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(means, stds)
    ])
    dataset = ADNIDataset('adni', csv, imagesFolder, transform = transforms, target_transform = ToLabelOutput(), truthLabel = truthLabel)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)
    
    featureExtract = False
    dropoutRate = 0.6
    auxEnabled = True
    usePretrained = True
    model_ft, input_size = initialize_model('inception', num_classes, featureExtract, dropoutRate, auxEnabled, use_pretrained=usePretrained)
    device = torch.device(deviceName if torch.cuda.is_available() else "cpu")
    model_state_dict = torch.load(weights, map_location=device)
    model_ft.load_state_dict(model_state_dict)

    model_ft.to(device)
    
    testY, predY = collectAllData(dataloader, model_ft, device, num_classes)
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    
    print(testY)
    print(preds)
    
    print(f"Accuracy: {acc}")
    print(f"B acc: {bacc}")
    f1 = sklearn.metrics.f1_score(testY, preds, average = None)
    print(f"F1: {f1}")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    
    print("Confusion matrix:")
    print(matrix)
    
    if num_classes == 3:
        lr_auc = roc_auc_score(testY, predY, multi_class = 'ovr', average = None)
        print(f"AUC ROC: {lr_auc}")
    
    if (num_classes == 2):
        testYMerged = testY.copy()
        testYMerged[testYMerged == 2] = 0
        lr_auc = roc_auc_score(testYMerged, torch.from_numpy(predY[:, 1]))
        print(f"AUC ROC: {lr_auc}")
        return
    
    print("------------")
    print("Merging 2 with 0")
    
    predY[:, 0] = predY[:, 0] + predY[:, 2]
    predY = predY[:, 0:2]
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    
    print(f"Accuracy merged: {acc}")
    print(f"B acc merged: {bacc}")
    
    print("Confusion matrix merged:")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    print(matrix)

In [49]:
for key in config:
    print(f"Ejecutando {key}")
    executeModel(mode = 'validation', **config[key])
    print("#################")

Ejecutando last_3_classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[2. 2. 2. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 2. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1.
 0. 0. 0. 0. 0. 2. 2. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 2. 1. 1. 1. 0. 0. 2. 1. 2. 2. 2. 2.
 2. 2. 2. 1. 1. 2. 2. 2. 1. 1. 1. 1. 2. 1. 1. 0. 2. 0. 1. 2. 2. 2. 2. 2.
 2. 0. 2. 0. 2. 0. 1. 1. 2. 1. 2. 2. 2. 1. 1. 1. 1. 2. 2. 1. 1. 1. 2. 2.
 2. 0. 2. 2. 2. 2. 2. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 2. 2. 2. 0. 0. 0. 1.
 1. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 2. 0. 0. 0.
 2. 0. 0. 0. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 2. 2. 1.
 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 0. 2. 0. 0. 0. 0. 0. 2. 2. 2. 2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 2. 2. 2. 2. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 2. 2. 2. 2. 2.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 2. 2. 2. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 2. 2.
 1. 2. 1. 1. 2. 2. 0. 0. 1. 1. 0. 0

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[1. 1. 1. 2. 2. 2. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 2. 1. 0. 0. 2. 1. 1. 1.
 1. 1. 1. 1. 1. 2. 2. 1. 0. 0. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 2. 1. 1. 1. 1. 1. 1. 2. 2. 0. 1. 1. 1. 1. 1. 0. 0. 2. 2. 2. 2. 2. 1. 1.
 1. 1. 0. 0. 2. 2. 2. 2. 2. 2. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 2. 1. 0. 1. 0. 2. 0. 0. 0. 0. 2. 2. 2. 2. 2.
 2. 1. 2. 2. 2. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 2. 2. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 2. 2. 2. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 0. 0. 0. 1. 2.
 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 2. 2. 2. 1. 2. 2. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 2. 2.
 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 0. 0. 0. 0. 1. 2. 2. 2. 2. 2. 1. 1. 1. 1.
 0. 2. 2. 1. 1. 2. 2. 2. 0. 0. 0. 0. 0. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 2. 0. 2. 1. 1. 1. 1.
 1. 1. 1. 2. 2. 2. 2. 1. 1. 1. 1. 1

...

num featurs2048
[2. 2. 2. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 2. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1.
 0. 0. 0. 0. 0. 2. 2. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 2. 1. 1. 1. 0. 0. 2. 1. 2. 2. 2. 2.
 2. 2. 2. 1. 1. 2. 2. 2. 1. 1. 1. 1. 2. 1. 1. 0. 2. 0. 1. 2. 2. 2. 2. 2.
 2. 0. 2. 0. 2. 0. 1. 1. 2. 1. 2. 2. 2. 1. 1. 1. 1. 2. 2. 1. 1. 1. 2. 2.
 2. 0. 2. 2. 2. 2. 2. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 2. 2. 2. 0. 0. 0. 1.
 1. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 2. 0. 0. 0.
 2. 0. 0. 0. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 2. 2. 1.
 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 0. 2. 0. 0. 0. 0. 0. 2. 2. 2. 2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 2. 2. 2. 2. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 2. 2. 2. 2. 2.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 2. 2. 2. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 2. 2.
 1. 2. 1. 1. 2. 2. 0. 0. 1. 1. 0. 0

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[1. 1. 1. 2. 2. 2. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 2. 1. 0. 0. 2. 1. 1. 1.
 1. 1. 1. 1. 1. 2. 2. 1. 0. 0. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 2. 1. 1. 1. 1. 1. 1. 2. 2. 0. 1. 1. 1. 1. 1. 0. 0. 2. 2. 2. 2. 2. 1. 1.
 1. 1. 0. 0. 2. 2. 2. 2. 2. 2. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 2. 1. 0. 1. 0. 2. 0. 0. 0. 0. 2. 2. 2. 2. 2.
 2. 1. 2. 2. 2. 2. 1. 1. 2. 1. 1. 2. 2. 2. 0. 2. 2. 2. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 2. 2. 2. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 0. 0. 0. 1. 2.
 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 2. 2. 2. 1. 2. 2. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 2. 2.
 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 0. 0. 0. 0. 1. 2. 2. 2. 2. 2. 1. 1. 1. 1.
 0. 2. 2. 1. 1. 2. 2. 2. 0. 0. 0. 0. 0. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 2. 0. 2. 1. 1. 1. 1.
 1. 1. 1. 2. 2. 2. 2. 1. 1. 1. 1. 1

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[0. 0. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 2. 2. 2.
 2. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 2. 2.
 2. 2. 2. 1. 1. 2. 2. 2. 0. 1. 0. 0. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 0. 2. 2. 2. 2. 1. 2. 2. 2.
 2. 0. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2.
 0. 0. 1. 0. 0. 2. 2. 1. 2. 1. 1. 1. 1. 2. 2. 2. 1. 1. 1. 2. 2. 2. 1. 2.
 2. 2. 0. 0. 0. 0. 0. 2. 1. 2. 2. 2. 1. 2. 1. 2. 1. 1. 2. 0. 2. 2. 2. 1.
 1. 1. 1. 0. 2. 0. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 2. 0. 0. 0. 0. 2. 0.
 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 2. 1. 1. 0. 0. 0. 2.
 2. 2. 2. 2. 2. 2. 1. 2. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 0. 0. 2. 2. 0.
 0. 1. 1. 1. 1. 0. 1. 2. 2. 2. 2. 2. 2. 2. 0. 2. 2. 2. 2. 2. 1. 1. 1. 1.
 2. 2. 2. 2. 1. 1. 0. 0. 1. 1. 1. 2. 2. 2. 1. 1. 1. 1. 2. 1. 2. 1. 2. 2.
 2. 2. 2. 2. 1. 1. 2. 2. 2. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 0. 0. 0. 0. 0. 0. 2. 2. 2. 0. 0

# Nuevos

In [12]:
configNew = {
  "last_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxlast_2classes_CN,AD_0_epoch42.pth",
    "num_classes": 2,
    "validationSet": "Muestra3700_80_10_10_CN,AD_dxlast_val.csv",
    "testSet": "Muestra3700_80_10_10_CN,AD_dxlast_test.csv",
    "truthLabel": "DX_last",
    "mean": 0.26039753425852596,
    "std": 0.4631627215324542
  },
  "sev_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxmost_severe_2classes_CN,AD_0_epoch12.pth",
    "num_classes": 2,
    "validationSet": "Muestra3700_80_10_10_CN,AD_dxmost_severe_val.csv",
    "testSet": "Muestra3700_80_10_10_CN,AD_dxmost_severe_test.csv",
    "truthLabel": "DX_most_severe",
    "mean": 0.2602020067344957,
    "std": 0.4630376667704176
  },
  "visit_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxvisit_2classes_CN,AD_0_epoch19.pth",
    "num_classes": 2,
    "validationSet": "Muestra3700_80_10_10_CN,AD_dxvisit_val.csv",
    "testSet": "Muestra3700_80_10_10_CN,AD_dxvisit_test.csv",
    "truthLabel": "DX",
    "mean": 0.2605264233400815,
    "std": 0.463615010031876
  },
  "last_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxlast_2classes_MCI,AD_0_epoch45.pth",
    "num_classes": 2,
    "validationSet": "Muestra3700_80_10_10_MCI,AD_dxlast_val.csv",
    "testSet": "Muestra3700_80_10_10_MCI,AD_dxlast_test.csv",
    "truthLabel": "DX_last",
    "mean": 0.2616675620409126,
    "std": 0.4638748014783555
  },
  "sev_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxmost_severe_2classes_MCI,AD_0_epoch17.pth",
    "num_classes": 2,
    "validationSet": "Muestra3700_80_10_10_MCI,AD_dxmost_severe_val.csv",
    "testSet": "Muestra3700_80_10_10_MCI,AD_dxmost_severe_test.csv",
    "truthLabel": "DX_most_severe",
    "mean": 0.2612156752725623,
    "std": 0.46354785230807755
  },
  "visit_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxvisit_2classes_MCI,AD_0_epoch1.pth",
    "num_classes": 2,
    "validationSet": "Muestra3700_80_10_10_MCI,AD_dxvisit_val.csv",
    "testSet": "Muestra3700_80_10_10_MCI,AD_dxvisit_test.csv",
    "truthLabel": "DX",
    "mean": 0.26129683687386074,
    "std": 0.46354744613985177
  }
}


In [51]:
for key in configNew:
    print(f"Ejecutando {key}")
    executeModel(mode = 'validation', **configNew[key])
    print("#################")

Ejecutando last_cn_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1
 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1 1

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 1.]
[0 0 1 1 1 1 1 0 0 0 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1
 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 0
 1 1 1 1 1

...

[1. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2.
 2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 2. 2. 1.
 1. 1. 1. 1. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 1. 1. 2. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 2. 2. 2. 2. 2.
 2. 2. 1. 2. 2. 2. 2. 1. 1. 2. 1. 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 2. 2. 2. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1.
 1. 1. 2. 2. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1.
 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 1. 2. 2. 1.
 1. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2.
 1. 1. 1. 1. 2. 2. 1. 1. 2. 2. 2. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1.
 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 2. 1. 1. 2. 2.
 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 1. 2.]
[0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048


/Users/hugom/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 1. 1. 1. 1. 2. 2. 2. 2. 2.
 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 2. 2. 2. 2. 1. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1.
 2. 2. 1. 2. 1. 1. 1. 1. 2. 2. 2. 1. 1. 1. 2. 2. 2. 1. 2. 2. 2. 2. 1. 2.
 2. 2. 1. 2. 1. 2. 1. 1. 2. 2. 2. 2. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1.
 1. 1. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 2. 2.
 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 2. 2. 2. 1. 1. 1. 1. 1. 2. 2. 2. 1. 1.
 1. 1. 2. 1. 2. 1. 2. 2. 2. 2. 2. 2. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]
[0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 0 1 0 0
 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 1 

# Test

In [20]:
for key in config:
    print(f"Ejecutando {key}")
    executeModel(mode = 'test', **config[key])
    print("#################")

Ejecutando last_3_classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[2. 2. 2. 0. 0. 2. 2. 2. 0. 0. 1. 1. 1. 1. 0. 0. 0. 2. 2. 2. 1. 0. 0. 0.
 0. 2. 0. 2. 1. 1. 1. 1. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 0. 2. 0. 0. 2. 0.
 1. 1. 1. 0. 1. 1. 1. 2. 2. 0. 2. 2. 2. 2. 1. 0. 0. 0. 0. 2. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 2. 2. 0. 2. 1. 1. 2. 2. 1. 1. 1. 2. 2. 2. 1. 2.
 1. 2. 1. 0. 2. 2. 2. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2.
 1. 1. 1. 1. 2. 0. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 2. 0. 0.
 2. 2. 2. 2. 0. 0. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 0. 0.
 2. 1. 1. 2. 2. 1. 2. 1. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 1.
 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 2. 2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 0. 0. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 0. 0. 1. 2. 1. 1. 1. 2

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[2. 2. 2. 2. 0. 0. 2. 0. 0. 2. 1. 1. 1. 1. 0. 1. 1. 2. 2. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 0. 0. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 1. 1. 1. 1. 2. 2. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 2. 1. 1. 2. 0. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 2. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 2. 1. 1. 2. 0. 0. 0. 0.
 0. 0. 2. 2. 2. 2. 1. 2. 2. 2. 2. 1. 1. 1. 1. 2. 1. 1. 0. 0. 1. 1. 1. 1.
 2. 2. 2. 0. 1. 2. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 2. 1. 1. 1. 1. 0. 0. 2. 2. 2. 2. 1.
 2. 2. 2. 2. 2. 2. 0. 0. 2. 0. 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 0. 1. 2. 2. 2. 2. 2. 2. 2. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 2

...

num featurs2048
[2. 2. 2. 0. 0. 2. 2. 2. 0. 0. 1. 1. 1. 1. 0. 0. 0. 2. 2. 2. 1. 0. 0. 0.
 0. 2. 0. 2. 1. 1. 1. 1. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 0. 2. 0. 0. 2. 0.
 1. 1. 1. 0. 1. 1. 1. 2. 2. 0. 2. 2. 2. 2. 1. 0. 0. 0. 0. 2. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 2. 2. 0. 2. 1. 1. 2. 2. 1. 1. 1. 2. 2. 2. 1. 2.
 1. 2. 1. 0. 2. 2. 2. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2.
 1. 1. 1. 1. 2. 0. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 2. 0. 0.
 2. 2. 2. 2. 0. 0. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 0. 0.
 2. 1. 1. 2. 2. 1. 2. 1. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 1.
 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 2. 2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 0. 0. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 0. 0. 1. 2. 1. 1. 1. 2

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[2. 2. 2. 2. 0. 0. 2. 0. 0. 2. 1. 1. 1. 1. 0. 1. 1. 2. 2. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 0. 0. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 1. 1. 1. 1. 2. 2. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 2. 1. 1. 2. 0. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 2. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 2. 1. 1. 2. 0. 0. 0. 0.
 0. 0. 2. 2. 2. 2. 1. 2. 2. 2. 2. 1. 1. 1. 1. 2. 1. 1. 0. 0. 1. 1. 1. 1.
 2. 2. 2. 0. 1. 2. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 2. 1. 1. 1. 1. 0. 0. 2. 2. 2. 2. 1.
 2. 2. 2. 2. 2. 2. 0. 0. 2. 0. 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 0. 1. 2. 2. 2. 2. 2. 2. 2. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 2

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[2. 2. 0. 2. 0. 0. 2. 2. 2. 1. 1. 2. 2. 2. 2. 2. 2. 2. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 2. 2. 2. 2. 2. 1. 1. 1. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 2. 2. 2. 2. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 2. 2. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 0. 2. 2. 2. 2. 1. 1. 2. 0. 0. 0.
 2. 1. 2. 2. 2. 2. 0. 0. 0. 1. 1. 2. 2. 2. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 2. 1. 1. 2. 2. 2. 2. 1. 1. 1. 1. 2. 2. 1. 2. 0. 0. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 0. 0. 0. 1. 1. 1. 1. 2. 2. 2. 0. 0. 0. 0. 2. 2. 2. 2.
 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 2. 2. 1. 0. 1. 2. 2. 2. 2. 2.
 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 2. 2. 2. 2. 1. 1. 1. 0. 0. 2. 2. 0.
 0. 2. 2. 0. 2. 0. 2. 2. 2. 2. 0. 2. 0. 0. 1. 0. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 2. 2. 2. 1. 1. 1. 1. 1. 2. 1. 1. 1. 2. 2.
 2. 0. 0. 0. 0. 2. 2. 2. 1. 1. 1. 1. 2. 2. 2. 2. 1. 2. 0. 2. 2. 2. 2. 2.
 2. 1. 1. 2. 1. 1. 1. 1. 1. 0. 1. 2. 2. 2. 2. 1. 1. 1. 0. 0. 0. 0. 0. 2.
 2. 2. 1. 1. 1. 1. 2. 2. 2. 1. 2. 2

In [21]:
for key in configNew:
    print(f"Ejecutando {key}")
    executeModel(mode = 'test', **configNew[key])
    print("#################")

Ejecutando last_cn_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0 0 0 0 1 1 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 0 1 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 1 0 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048
[0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 0.]
[0 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1
 1 1 1 0 1 1 1 1 0 0 0 0 1

...

[2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 2. 2. 2. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1.
 1. 2. 1. 1. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2.
 2. 1. 1. 1. 1. 1. 1. 2. 1. 1. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 1. 1. 1. 1.
 2. 1. 1. 1. 1. 1. 1. 2. 2. 2. 1. 2. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 2. 1. 1. 1. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2.
 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.
 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 1. 2.
 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 2. 2. 2. 2. 2. 1. 2.]
[0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 1 1 1 1 1 0 

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


num featurs2048


/Users/hugom/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[2. 2. 2. 2. 2. 2. 1. 1. 2. 2. 2. 2. 2. 2. 2. 1. 1. 2. 2. 2. 2. 2. 1. 1.
 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2.
 2. 1. 1. 2. 2. 1. 2. 2. 2. 2. 1. 1. 2. 2. 2. 1. 2. 1. 1. 2. 2. 2. 2. 1.
 1. 1. 1. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2.
 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 2. 2. 1. 1. 2. 2. 2. 2. 2. 1. 2. 2. 2.
 2. 2. 2. 1. 1. 1. 1. 2. 2. 2. 2. 2. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1.
 1. 1. 2. 1. 1. 1. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 2. 2. 2. 1. 2. 2. 2.
 2. 2. 2. 2. 1. 1. 2. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 1. 1. 1. 2. 2. 2. 1.
 1. 1. 1. 2. 2. 2. 1. 2. 2. 2. 1. 2. 2. 2. 1. 1. 2. 2. 2. 2. 1. 1. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2.]
[0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1
 0 1 1 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 0 0 1 1
 1 1 1 1 1 1 0 0

# Test